In [1]:
#Import needed libraries
import pandas as pd
import numpy as np
import scipy
import scipy.stats
import random
import os
import pickle
import theano
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.cuda as cuda
import matplotlib.pyplot as plt
from skorch.net import NeuralNetClassifier
import torch.utils.data as Data
from scipy import stats

# CUDA initializing

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # If CUDA is available => cuda:0 prints
print(device)

# Import data

In [ ]:
# Buid the feature matrix
data = pd.read_csv('/home/xsong/Alma/2017---Deep-learning-yeast-UTRs/Data/Random_UTRs.csv')

In [ ]:
print(data.shape())

## One-hot encoding of the sequences.

i.e. we're converting the sequences from being represented as a 50 character string of bases to a 4x50 matrix of 1's and 0's, with each row corresponding to a base and every column a position in the UTR.

In [ ]:
# From the work of Cuperus et al.
# one hot encoding of UTRs
# X = one hot encoding matrix
# Y = growth rates

def one_hot_encoding(df, seq_column, expression):

    bases = ['A','C','G','T']
    base_dict = dict(zip(bases,range(4))) # {'A' : 0, 'C' : 1, 'G' : 2, 'T' : 3}

    n = len(df)
    
    # length of the UTR sequence
    # we also add 10 empty spaces to either side
    total_width = df[seq_column].str.len().max() + 20
    
    # initialize an empty numpy ndarray of the appropriate size
    X = np.zeros((n, 1, 4, total_width))
    
    # an array with the sequences that we will one-hot encode
    seqs = df[seq_column].values
    
    # loop through the array of sequences to create an array that keras will actually read
    for i in range(n):
        seq = seqs[i]
        
        # loop through each individual sequence, from the 5' to 3' end
        for b in range(len(seq)):
            # this will assign a 1 to the appropriate base and position for this UTR sequence
            X[i, 0, base_dict[seq[b]], int(b + round((total_width - len(seq))/2.))] = 1.
    
        # keep track of where we are
        if (i%100000)==0:
            print(i),
        
    X = X.astype(theano.config.floatX)
    Y = np.array(df[expression].values,
                   dtype = theano.config.floatX)[:, np.newaxis]
    
    return X, Y, total_width

In [ ]:
X, Y, total_width = one_hot_encoding(data, 'UTR', 'growth_rate')

In [ ]:
print(X.shape)

In [ ]:
X_torch = torch.from_numpy(X).float().cuda() #change to torch and upload to CUDA
Y_torch = torch.from_numpy(Y).float().cuda() #change to torch and upload to CUDA

In [ ]:
print(X_torch)

## Generate different data sets

In [ ]:
# a sorted numpy array of UTR indexes, from least reads to most reads
sorted_inds = data.sort_values('t0').index.values
train_inds = sorted_inds[:int(0.95*len(sorted_inds))] # 95% of the data as the training set
test_inds = sorted_inds[int(0.95*len(sorted_inds)):] # UTRs with most reads at time point 0 as the test set

# set the seed before randomly shuffling the data
seed = 0.5
random.shuffle(train_inds, lambda :seed)

# Generate Model

I need to figure out how to make the dropout happen and Flatten. 
How do hidden units work in fully connected layers?

## Buid the neural network

In [ ]:
size=1
batch_size=10
class Net(nn.Module):
    def __init__(self, x):
        super(Net, self).__init__()
        # input channel, output channels = number of filters, convolution kernel size
        # kernel
        self.conv1 = nn.Conv2d(1, size, [4,13])
        self.conv2 = nn.Conv2d(1, size, [1,13])
        self.conv3 = nn.Conv2d(1, size, [1,13])
        self.fc1 = nn.Linear(34, 120)
        self.lin_out1 = nn.Linear(120, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #print('conv1',x.size())
        x = F.relu(self.conv2(x))
        #print('conv2',x.size())
        x = F.relu(self.conv3(x))
        #print('conv3',x.size())
        x = F.relu(self.fc1(x))
        #print('fc1',x.size())
        x = self.lin_out1(x)
        #print('lol1',x.size())
        #x = nn.Dropout(p=0.15)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net(data)
net = net.to(device)
print(net)
next(net.parameters()).is_cuda

In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) 

## Training & Cross Validation


In [ ]:
def eval(net, data):
        net.eval()
        ######EVALUATION STEP#################
        #same as during training step, need to do batch training
        correct_count = 0.
        total_count = 0.
        for i, (dev_data_x,dev_data_y) in enumerate(dev_loader):
            dev_data_y.requires_grad=False
            dev_data_y = dev_data_y[:,0]   # resize the value as vector
            pred = net(dev_data_x)
            pred = pred[:,0,0,0]   #resize the output as vector
            fit= loss_func(pred, dev_data_y)
        return loss

In [ ]:
# Choice of optimizer & loss function => MSE 
# Using backpropagation

# Initiate the hyperparameters
number_epochs = 20
track_loss = []
loss_func = nn.MSELoss().cuda()
k_fold = 10

# Define dataset and initialize mini-batch data
x = X_torch[train_inds]
y = Y_torch[train_inds]
num_data = x.shape[0]
num_dev_data = int(num_data/k_fold) #evalutation data amount
fit_store = np.tile(0,(k_fold,number_epochs))
fit_store = torch.from_numpy(fit_store).float().cuda()
batch_size=32

print(num_data - num_dev_data, num_dev_data)

In [ ]:
#Training with crossvalidation
best_save=np.zeros((k_fold), float)
for split in range(k_fold):
    train_index = np.empty
    dev_index = np.empty
    dev_index = list(range(num_dev_data*split, num_dev_data*(split+1)))
    if split == 0:
        train_index = np.array(range(num_dev_data*(split+1),num_dev_data*k_fold))
    else:
        train_index = np.array(range(0, num_dev_data*split))
        train_index = np.append(train_index, np.array(range(num_dev_data*(split+1),num_dev_data*k_fold)))
        
    dev_data_x = x[dev_index]      #evalutation data
    dev_data_y = y[dev_index]      #evalutation data

    train_data_x = x[train_index]
    train_data_y = y[train_index]
    
    train_dataset = Data.TensorDataset(train_data_x, train_data_y)
    train_loader = Data.DataLoader(train_dataset, batch_size, shuffle=True)
    
    dev_dataset = Data.TensorDataset(dev_data_x, dev_data_y)
    dev_loader = Data.DataLoader(dev_dataset, batch_size, shuffle=True)  
    
    learning_rate = 0.005
    running_loss = 0.
    best_fit = 10
    optimizer = torch.optim.Adam(net.parameters(), learning_rate)

    for epoch in range(number_epochs): # loop over the dataset multiple time        
        #######TRAINING STEP##################
        j=1
        for i, (train_data_x,train_data_y) in enumerate(train_loader):
            train_data_y.requires_grad=False
            train_data_y = train_data_y[:,0].float()                 # resize the value as vector
            optimizer.zero_grad()      # zero the parameter gradients
            output = net(train_data_x)
            output = output[:,0,0,0]   #resize the output as vector
            loss = loss_func(output, train_data_y)# compute the loss of the system
            loss.backward()            # start backward function
            optimizer.step()           # optimizing step
            running_loss += loss.item()
            if ((i+1)%1000)==0:
                print(epoch+1, i+1 ,running_loss / 1000)
                track_loss.append(running_loss / 1000)
                running_loss=0.
        fit = eval(net, dev_data_x)
        fit_store[split,epoch]=fit
        print(epoch+1,fit[0])
        if fit<best_fit:
            best_fit = fit
            print("SAVE FILE AS Model_training_28-06-18_" + str(split+1) + "_" + str(epoch+1))
            torch.save(net.state_dict(),"/home/xsong/Alma/Training/Model_training_01-07-18_" + str(split+1) + "_" + str(epoch+1) + ".pt")
            best_save(split) = epoch
        else:
            learning_rate *= 0.8
            optimizer = torch.optim.Adam(net.parameters(), learning_rate)
print('Finished Training')

In [ ]:
# Plotting of the loss function
time = np.array(range(0, number_epochs))
time = np.tile(time, (10,1))
fit_plot = fit_store.data.cpu().numpy()
print(fit_plot)
plt.plot(fit_plot)
plt.axis([1,20, 0.5, 2])
plt.ylabel('Value of the loss function')
plt.xlabel('Epochs')
plt.show()

In [ ]:
plt.plot(fit_plot_t)
plt.ylabel('Value of the loss function')
plt.xlabel('Epochs')
#plt.show()
plt.savefig("/home/xsong/Desktop/test_loss.png", dpi = 300)

In [ ]:
## reorganize running_loss function
loss_plot = np.reshape(track_loss, (200,3))
loss_mean = loss_plot.mean(axis=1)
print(loss_mean)
loss_mean = np.reshape(loss_mean,(10,20))
loss_mean = np.transpose(loss_mean)
print(loss_mean)

In [ ]:
# Plotting of the loss function
time = np.array(range(0, number_epochs))
time = np.tile(time, (10,1))
#running_loss_plot = running_loss.data.cpu().numpy()
#running_loss_plot = np.ndarray.transpose(running_loss_plot())
plt.plot(loss_mean)
plt.ylabel('Value of the loss function')
plt.xlabel('Epochs')
plt.axis([1,20, 1.195, 1.265])
#plt.show()
plt.savefig("/home/xsong/Desktop/running.png", dpi = 300)

## Verification


During testing time, I load all ten models and keep them in a list. When making a prediction on a single instance, I apply each of the 10 models to give scores, and add the predicted scores up to make a single prediction. I do argmax over the summed scores to make the final prediction.


In [ ]:
val_x = X_torch[test_inds]
val_y = Y_torch[test_inds]

In [ ]:
models = []
best_save = [9,32,49,7,36,24,36,4,49,4]
for split in range(k_fold):
    model = Net(data)
    model.load_state_dict(torch.load("/home/xsong/Alma/Training/50_epochs_training/Model_training_01-07_" + str(split+1) + "_" + str(best_save[split]) + ".pt"))
    models.append(net)

In [ ]:
test_dataset = Data.TensorDataset(val_x, val_y)
test_loader = Data.DataLoader(test_dataset, batch_size, shuffle=True)  
def test(net, data):
        fit = []
        fit = torch.FloatTensor(fit).cuda()
        for i, (val_x,val_y) in enumerate(test_loader):
            val_y.requires_grad=False
            val_y = val_y[:,0] # resize the value as vector
            pred = []
            pred = torch.FloatTensor(pred).cuda()
            preds = []
            preds = torch.FloatTensor(preds).cuda()
            for model in models:
                pred = model(val_x)
                pred = pred[:,0,0,0]
                print(pred)
                preds += pred
            fit += loss_func(preds, val_y)
            slope, intercept, r_value, p_value, std_err += stats.linregress(preds,val_y)
            print(fit,r_value)
        return fit,preds,r_value

In [ ]:
fit,preds,r_value = test(models, data)
print(fit,preds,r_value)

## Plot predictions vs data

In [ ]:
# data
x = Y_pred.flatten()
y = Y[test_inds].flatten()

# calculate R^2
r2 = scipy.stats.pearsonr(x, y)[0]**2


g = sns.jointplot(x,
                  y,
                  stat_func = None,
                  kind = 'scatter',
                  s = 5,
                  alpha = 0.1,
                  size = 5)

g.ax_joint.set_xlabel('Predicted log$_2$ Growth Rate')
g.ax_joint.set_ylabel('Measured log$_2$ Growth Rate')


text = "R$^2$ = {:0.2}".format(r2)
plt.annotate(text, xy=(-5.5, 0.95), xycoords='axes fraction')

plt.title("CNN predictions vs. test set", x = -3, y = 1.25)